In [1]:
import pandas as pd
import pickle
import sqlalchemy
import pymysql
import os
from urllib.parse import quote
from cleaningScript import cleanDatav2
from dotenv import load_dotenv
from sklearn.ensemble import RandomForestRegressor

In [2]:
load_dotenv()
password = os.getenv('DB_PASSWORD')

In [3]:
DBengine = sqlalchemy.create_engine("mysql+pymysql://student:{}@localhost:3306/dublinbus".format(quote(password)))

In [4]:
dfBase = pd.read_sql('call dublinbus.all_lines_directions();', DBengine) #using stored procedure

In [5]:
lines = dfBase.values.tolist()

In [ ]:
for line,direction in lines:
    print('route:',line, direction)
    df = cleanDatav2(line,direction)
    print('loaded!')
    df['weekday'] = df.date.dt.weekday
    df['month'] = df.date.dt.month
    df['hour'] = (df.stopActualArr//3600)
    df = df.drop(columns = ['stopActualArr','rain','temp','pressure','dwelltime'])
    df.hour %= 24
    df.month = df.month.astype('category')
    df.hour = df.hour.astype('category')
    df.weekday = df.weekday.astype('category')
    monthDummies = pd.get_dummies(df.month, prefix='m', drop_first=True)
    hourDummies = pd.get_dummies(df.hour,prefix='h', drop_first=True)
    dayDummies = pd.get_dummies(df.weekday, prefix='d', drop_first=True)
    df = pd.concat([df,monthDummies,hourDummies,dayDummies], axis=1)
    df.drop(columns=['month','hour','weekday'], inplace=True)
    Xfeatures = df.columns[df.columns != 'journeytime']
    Xfeatures = Xfeatures.drop(['date','tripid','humidity'])
    X = df[Xfeatures]
    y = df.journeytime
    print('ready!')
    rfc = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=1, n_jobs=-1)
    rfc.fit(X,y)
    print('saving!')
    with open('models/model{}_{}.pkl'.format(line,direction),'wb') as file:
        pickle.dump(rfc,file, pickle.HIGHEST_PROTOCOL)

route: 14 1
loaded!
ready!
